In [10]:
import folium
from folium.plugins import Fullscreen
import geopandas as gpd
import xyzservices as xyz
import os
import re
import importlib
import src.models.base as base
importlib.reload(base)
from src.models.base import get_conn
import pandas as pd
import elkai
pd.set_option('display.expand_frame_repr', False) # display full data in terminal

engine, Session = get_conn('climb_wise_local')

In [11]:
with Session() as session:
    waypoints = gpd.read_postgis("SELECT * FROM waypoints WHERE notes  = '2023 2024 Van Trip'", con=engine, geom_col='geometry')

def traveling_salesman(waypoints):
    # Move 'Denver Colorado' to the first position in waypoints
    denver = waypoints[waypoints.title == 'Denver Colorado']
    not_denver = waypoints[waypoints.title != 'Denver Colorado']
    places = pd.concat([denver, not_denver]).reset_index(drop=True)

    waypoints_dict = {}
    for _, row in places.iterrows():
        waypoints_dict[row.title] = (row.latitude, row.longitude)

    runs = 1

    return elkai.Coordinates2D(waypoints_dict).solve_tsp(runs=runs)

solved_cities = traveling_salesman(waypoints)

In [ ]:
# Create map

def escape_placeholders(url, placeholders):
    # Escape curly braces around the specified placeholders
    for placeholder in placeholders:
        url = re.sub(rf'\{{({placeholder})\}}', r'{{\1}}', url)
    return url

def add_open_street_layer(map):
    folium.TileLayer('OpenStreetMap', name='Open Street').add_to(map)

def add_open_topo_layer(map):
    folium.TileLayer(
        tiles='https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png',
        attr='Map data: &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors, <a href="http://viewfinderpanoramas.org">SRTM</a> | Map style: &copy; <a href="https://opentopomap.org">OpenTopoMap</a> (<a href="https://creativecommons.org/licenses/by-sa/3.0/">CC-BY-SA</a>)',
        name='Open Topo'
    ).add_to(map)

def add_watercolor_layer(map):
    water_color = xyz.providers.Stadia.StamenWatercolor
    stadia_api_key = os.getenv('STADIA_API_KEY')
    base_url = water_color.url
    water_color_url = escape_placeholders(base_url, ['z', 'x', 'y'])
    substituted_url = water_color_url.format(variant='stamen_watercolor', ext='jpg') + '?api_key=' + stadia_api_key
    stadia_attr = water_color.attribution
    folium.TileLayer(
        tiles=substituted_url,
        attr=stadia_attr,
        name='Watercolor',
        opacity=0.85
    ).add_to(map)

def add_stamen_sattelite_layer(map):
    sattelite = xyz.providers.Stadia.AlidadeSatellite
    stadia_api_key = os.getenv('STADIA_API_KEY')
    base_url = sattelite.url
    sattelite_url = escape_placeholders(base_url, ['z', 'x', 'y', 'r'])
    substituted_url = sattelite_url.format(variant='alidade_satellite', ext='jpg') + '?api_key=' + stadia_api_key
    stadia_attr = sattelite.attribution
    folium.TileLayer(
        tiles=substituted_url,
        attr=stadia_attr,
        name='Sattelite'
    ).add_to(map)

def add_sattelite_layer(map):
    folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Satellite'
       ).add_to(map)

def add_van_trip_waypoints(map):
    waypoints_layer = folium.FeatureGroup(name='Van Trip', overlay=True)
    for _, point in waypoints.iterrows():
        folium.Marker(
            location=[point.latitude, point.longitude],
            tooltip=point.title,
            icon=folium.Icon(icon='fa-van-shuttle', prefix='fa')
        ).add_to(waypoints_layer)
    waypoints_layer.add_to(map)

def add_gps_trails(map):
    trails_layer = folium.FeatureGroup(name='GPS Tracks', overlay=True, show=False)
    trail_gdf = gpd.read_postgis("SELECT * FROM trails WHERE notes IS NULL", con=engine, geom_col='geometry')
    trail_gdf["geometry"] = trail_gdf.geometry.simplify(tolerance=0.0001, preserve_topology=False)

    for _, trail in trail_gdf.iterrows():
        folium.GeoJson(trail['geometry'], name=trail['title'], tooltip=trail['title'], color='red').add_to(trails_layer)
    trails_layer.add_to(map)

def add_intl_gps_data(map):
    intl_travel_layer = folium.FeatureGroup(name='International Travel', overlay=True, show=False)
    trail_gdf = gpd.read_postgis("SELECT * FROM trails WHERE notes = 'International Travel'", con=engine, geom_col='geometry')
    intl_waypoint_geo_df = gpd.read_postgis("SELECT * FROM waypoints WHERE notes = 'International Travel'", con=engine, geom_col='geometry')
    
    for _, trail in trail_gdf.iterrows():
        folium.GeoJson(trail['geometry'], name=trail['title'], tooltip=trail['title'], color='red').add_to(intl_travel_layer)

    for _, point in intl_waypoint_geo_df.iterrows():
        folium.Marker(
            location=[point.latitude, point.longitude],
            tooltip=point.title,
            icon=folium.Icon(icon='fa-plane', prefix='fa', color='lightred')
        ).add_to(intl_travel_layer)
    intl_travel_layer.add_to(map)

def add_tsp_layer(map):
    ordered_indices = [waypoints[waypoints.title == title].index[0] for title in solved_cities]
    ordered_waypoints = waypoints.loc[ordered_indices].reset_index(drop=True)

    coords = [[w.latitude, w.longitude] for _, w in ordered_waypoints.iterrows()]

    tsp_layer = folium.FeatureGroup(name='Van Trip TSP', overlay=True, show=False)
    folium.PolyLine(
        locations=coords,
        color='blue',
        weight=2,
        opacity=1
    ).add_to(tsp_layer)
    tsp_layer.add_to(map)

def add_map_tools(map):
    Fullscreen(position='topright', force_separate_button=True).add_to(map)
    folium.LayerControl().add_to(map)

def main():
    center = [44.2, -102]
    map = folium.Map(tiles=None, location=center, zoom_start=4, control_scale=True)
    add_sattelite_layer(map)
    # add_stamen_sattelite_layer(map)
    add_open_street_layer(map)
    add_open_topo_layer(map)
    add_watercolor_layer(map)

    add_van_trip_waypoints(map)
    add_tsp_layer(map)
    add_gps_trails(map)
    add_intl_gps_data(map)

    add_map_tools(map)
    return map

map = main()
map.save('van_trip_map.html')
map